# Model Usage Demonstration (Unseen NASA Data)

This notebook demonstrates how the trained machine learning models are used in a real-world scenario. We apply all trained models to an unseen dataset from the NASA Exoplanet Archive and observe:

- Predicted planet disposition
- Prediction confidence
- Differences between models
- Which model performs most reliably

This simulates how such a system would be used in practice.

In [26]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

## Paths and Constants

In [27]:
DATA_PATH = "../data/processed/nasa_external_processed.csv"
MODELS_PATH = Path("../models")
RESULTS_PATH = Path("../Results/nasa_model_predictions.csv")

RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)

LABEL_MAP = {
    0: "FALSE POSITIVE",
    1: "CANDIDATE",
    2: "CONFIRMED"
}

## Load Processed External Dataset

In [28]:
df_usage = pd.read_csv(DATA_PATH)
df_usage.head()

,kepoi_name,kepler_name,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,...,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,true_label
0,K00752.01,Kepler-227 b,10797460,1.000,0,0,0,0,9.488036,2.775000e-05,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,2
1,K00752.02,Kepler-227 c,10797460,0.969,0,0,0,0,54.418383,2.479000e-04,...,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,2
2,K00753.01,NaN,10811496,0.000,0,0,0,0,19.899140,1.494000e-05,...,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436,1
3,K00754.01,NaN,10848459,0.000,0,1,0,0,1.736952,2.630000e-07,...,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,0
4,K00755.01,Kepler-664 b,10854555,1.000,0,0,0,0,2.525592,3.761000e-06,...,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,2


## Separate Identifiers and Features

In [29]:
IDENTIFIER_COLS = ["kepoi_name", "kepler_name"]

identifiers = df_usage[IDENTIFIER_COLS].copy()

X_usage_raw = df_usage.drop(columns=IDENTIFIER_COLS, errors="ignore")

# Drop target if present
if "true_label" in X_usage_raw.columns:
    X_usage_raw = X_usage_raw.drop(columns=["true_label"])

print("Raw feature shape:", X_usage_raw.shape)

Raw feature shape: (9564, 44)


## Load Training Feature Schema

Models require the exact same feature set and order as used during training.

In [33]:
training_features = joblib.load(MODELS_PATH / "training_features.pkl")

print("Number of training features:", len(training_features))

Number of training features: 31


## Align External Data to Training Features

In [34]:
# Replace inf with NaN
X_usage_raw = X_usage_raw.replace([np.inf, -np.inf], np.nan)

# Align columns (missing engineered features become NaN)
X_usage_aligned = X_usage_raw.reindex(columns=training_features)

# Fill remaining NaNs safely
X_usage_aligned = X_usage_aligned.fillna(0)

# Final safety check
assert list(X_usage_aligned.columns) == training_features
assert X_usage_aligned.isnull().sum().sum() == 0

print("Aligned feature matrix shape:", X_usage_aligned.shape)

Aligned feature matrix shape: (9564, 31)


## Load Scaler (Used During Training)

In [35]:
scaler = joblib.load(MODELS_PATH / "standard_scaler.pkl")

X_usage_scaled = scaler.transform(X_usage_aligned)

print("Scaled feature matrix created.")

Scaled feature matrix created.


## Load All Trained Models

In [36]:
models = {
    "Logistic Regression": joblib.load(MODELS_PATH / "baseline_logistic_regression.pkl"),
    "Random Forest": joblib.load(MODELS_PATH / "random_forest_model.pkl"),
    "Gradient Boosting": joblib.load(MODELS_PATH / "gradient_boosting_model.pkl"),
    "Extra Trees": joblib.load(MODELS_PATH / "extra_trees.pkl"),
    "AdaBoost": joblib.load(MODELS_PATH / "adaboost.pkl"),
    "Gaussian Process": joblib.load(MODELS_PATH / "gaussian_process.pkl"),
    "K Nearest Neighbors": joblib.load(MODELS_PATH / "knn_model.pkl"),
    "Naive Bayes": joblib.load(MODELS_PATH / "naive_bayes_model.pkl"),
    "SVM RBF": joblib.load(MODELS_PATH / "svm_rbf_model.pkl"),
    "MLP Neural Network": joblib.load(MODELS_PATH / "mlp_neural_network_model.pkl"),
    "Ridge Classifier": joblib.load(MODELS_PATH / "ridge_classifier.pkl"),
}

print(f"Loaded {len(models)} models.")

Loaded 11 models.


## Define Models That Require Scaling

In [37]:
scaled_models = {
    "Logistic Regression",
    "SVM RBF",
    "K Nearest Neighbors",
    "MLP Neural Network",
    "Ridge Classifier"
}

## Prediction Helper Function

In [38]:
def predict_with_model(model, X):
    preds = model.predict(X)

    if hasattr(model, "predict_proba"):
        probs = model.predict_proba(X)
        confidence = probs.max(axis=1)
    else:
        confidence = np.full(len(preds), np.nan)

    return preds, confidence

## Run Predictions on External NASA Data

In [39]:
all_results = identifiers.copy()

for model_name, model in models.items():

    X_input = X_usage_scaled if model_name in scaled_models else X_usage_aligned

    preds, conf = predict_with_model(model, X_input)

    all_results[f"{model_name}_prediction"] = [LABEL_MAP[p] for p in preds]
    all_results[f"{model_name}_confidence"] = conf

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 400 out of 400 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 400 out of 400 | elapsed:    0.0s finished
c:\Users\ibaan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but GaussianProcessClassifier was fitted without feature names
  warnings.warn(
c:\Users\ibaan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but GaussianProcessClassifier was fitted without feature names
  warnings.warn(
c:\U

In [40]:
all_results.head(10)

,kepoi_name,kepler_name,Logistic Regression_prediction,Logistic Regression_confidence,Random Forest_prediction,Random Forest_confidence,Gradient Boosting_prediction,Gradient Boosting_confidence,Extra Trees_prediction,Extra Trees_confidence,...,K Nearest Neighbors_prediction,K Nearest Neighbors_confidence,Naive Bayes_prediction,Naive Bayes_confidence,SVM RBF_prediction,SVM RBF_confidence,MLP Neural Network_prediction,MLP Neural Network_confidence,Ridge Classifier_prediction,Ridge Classifier_confidence
0,K00752.01,Kepler-227 b,FALSE POSITIVE,0.998363,FALSE POSITIVE,0.515,FALSE POSITIVE,0.967674,FALSE POSITIVE,0.6125,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466238,FALSE POSITIVE,0.941943,CANDIDATE,NaN
1,K00752.02,Kepler-227 c,FALSE POSITIVE,0.997184,FALSE POSITIVE,0.520,FALSE POSITIVE,0.895653,FALSE POSITIVE,0.6375,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466239,FALSE POSITIVE,0.920935,CANDIDATE,NaN
2,K00753.01,NaN,FALSE POSITIVE,0.999829,CONFIRMED,0.495,CONFIRMED,0.998406,CONFIRMED,0.3975,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466225,FALSE POSITIVE,0.882897,FALSE POSITIVE,NaN
3,K00754.01,NaN,FALSE POSITIVE,0.999282,CONFIRMED,0.555,CONFIRMED,0.999603,CONFIRMED,0.7775,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466226,FALSE POSITIVE,0.601587,FALSE POSITIVE,NaN
4,K00755.01,Kepler-664 b,FALSE POSITIVE,0.996542,FALSE POSITIVE,0.520,FALSE POSITIVE,0.961932,FALSE POSITIVE,0.6175,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466220,FALSE POSITIVE,0.947411,CANDIDATE,NaN
5,K00756.01,Kepler-228 d,FALSE POSITIVE,0.997221,FALSE POSITIVE,0.525,FALSE POSITIVE,0.958771,FALSE POSITIVE,0.6050,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466219,FALSE POSITIVE,0.945561,CANDIDATE,NaN
6,K00756.02,Kepler-228 c,FALSE POSITIVE,0.995521,FALSE POSITIVE,0.505,FALSE POSITIVE,0.934694,FALSE POSITIVE,0.6075,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466219,FALSE POSITIVE,0.926616,CANDIDATE,NaN
7,K00756.03,Kepler-228 b,FALSE POSITIVE,0.993845,FALSE POSITIVE,0.505,FALSE POSITIVE,0.887903,FALSE POSITIVE,0.6075,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466219,FALSE POSITIVE,0.899526,CANDIDATE,NaN
8,K00114.01,NaN,FALSE POSITIVE,0.634284,CONFIRMED,0.720,CONFIRMED,0.999927,CONFIRMED,0.8900,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466215,CONFIRMED,0.490968,CANDIDATE,NaN
9,K00757.01,Kepler-229 c,FALSE POSITIVE,0.998674,FALSE POSITIVE,0.510,FALSE POSITIVE,0.786307,FALSE POSITIVE,0.5950,...,CONFIRMED,0.571429,CANDIDATE,1.0,CANDIDATE,0.466252,FALSE POSITIVE,0.931446,CANDIDATE,NaN


In [41]:
# Majority Vote Across Models
prediction_cols = [c for c in all_results.columns if c.endswith("_prediction")]

all_results["majority_vote"] = (
    all_results[prediction_cols]
    .mode(axis=1)[0]
)

all_results[["kepoi_name", "kepler_name", "majority_vote"]].head()


,kepoi_name,kepler_name,majority_vote
0,K00752.01,Kepler-227 b,FALSE POSITIVE
1,K00752.02,Kepler-227 c,FALSE POSITIVE
2,K00753.01,NaN,CONFIRMED
3,K00754.01,NaN,CONFIRMED
4,K00755.01,Kepler-664 b,FALSE POSITIVE


## Evaluation

The result is compared to nasa KOI dataset using the kepoi_name and all the results are wrong so therefore re-training must be done (03_label_alignment_and_retraining).